In [17]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from numpy import array

from auto_encoder import NeuralNetwork

In [18]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [19]:
project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\ratings_small.csv")
data = list(csv.reader(open(data_path)))

data = data[1:]
data = np.array(data)
data[1]

array(['1', '1029', '3.0', '1260759179'], dtype='<U10')

In [20]:
movies = []
for i in range(len(column(data,1))):
    movies.append(data[i][1])
    
movies = set(movies)
movies = sorted(movies, key=lambda x: int(x), reverse=False)

In [21]:
users = []
for i in range(len(column(data,0))):
    users.append(data[i][0])

users = set(users)
users = sorted(users, key=lambda x: int(x), reverse=False)

In [22]:
user_movie = np.zeros((len(users),len(movies)), dtype=float)
print(np.shape(user_movie))

(671, 9066)


In [23]:
for i in range(len(data)):
    user_movie[users.index(data[i][0])][movies.index(data[i][1])] = data[i][2]
print(np.shape(user_movie))

(671, 9066)


In [28]:
neuron_list = [9066,4533,4533,9066]
nn = NeuralNetwork(neuron_list)
print(np.shape(user_movie))
inp = [np.transpose(user_movie)]
err = nn.train(inp,inp, 1)
plt.plot(err)
plt.show()
print(err)

(671, 9066)
Epoch: 0
Est: [[    -0.          23612.27914433  17785.91958122 ...   9839.25029356
    2404.67855691  19853.38049417]
 [    -0.           8786.17052692  38671.36194711 ...     -0.
   14524.88049761 115152.22941036]
 [    -0.             -0.             -0.         ...   1410.4166365
   73112.13877789     -0.        ]
 ...
 [    -0.          88716.99288562   1500.21748601 ...    154.81983044
   48729.1973186   64966.93875166]
 [    -0.          15967.1454885      -0.         ...  18817.79760682
    1827.03098196  37498.94672488]
 [    -0.            413.03137939  14984.97817862 ... 100531.1210136
      -0.          75737.70517906]]
Out: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]]



ValueError: setting an array element with a sequence.